In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
dataset_path = '/content/gdrive/My Drive/Projects/Pytorch/Sentiment_analysis/'

In [3]:
import torch
from torchtext import data

SEED = 1234
import pandas as pd
import numpy as np
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import nltk

import random
from sklearn.metrics import classification_report

#import pyprind
%matplotlib inline 

In [4]:
import spacy
spacy_en = spacy.load('en')

In [5]:
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))

Cuda Status on system is True


Different filter sizes that we want in our CNN model. Filter size of 3 means that filter will focus on 3 consecutive words in the sentence

In [6]:
FILTER_SIZES = [3,4,5]

Tokenizing each sentence and adding PAD tokens if the length is less than the length of the filter.

In [7]:
def tokenizer(text):
    token = [t.text for t in spacy_en.tokenizer(text)]
    if len(token) < FILTER_SIZES[-1]:
        for i in range(0, FILTER_SIZES[-1] - len(token)):
            token.append('<PAD>')
    return token

In [8]:
TEXT = data.Field(tokenize = tokenizer, batch_first = True)


In [9]:
LABEL = data.LabelField(dtype = torch.long)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path=dataset_path, train="sst_train.csv", 
    validation="sst_dev.csv", test="sst_test.csv",format="csv", skip_header=True, 
    fields=[('text', TEXT), ('truth', LABEL)]
)

In [12]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of valid examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8544
Number of valid examples: 1101
Number of testing examples: 2210


In [13]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors(dataset_path+"glove.840B.300d.txt"), 
                 max_size=25000,unk_init = torch.Tensor.normal_)


100%|█████████▉| 2195820/2196017 [05:50<00:00, 6564.84it/s]

In [14]:
LABEL.build_vocab(train_data)

In [15]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key=lambda x: len(x.text),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=device)
LABEL.vocab.freqs

Counter({'1': 1092, '2': 2218, '3': 1624, '4': 2322, '5': 1288})

A CNN model is defined

1.   CNN filters are added on the text to extract features
2.   All of them are combined as passed through a FC layer



In [18]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

The best set of hyperparameters

In [19]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
OUTPUT_DIM = 5
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,511,605 trainable parameters


In [25]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.8430, -0.0559, -0.0837,  ...,  0.9208, -0.2708, -0.4322],
        [ 0.4218,  0.2891,  0.6224,  ..., -0.0994, -0.3216, -0.2066]],
       device='cuda:0')

In [24]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [23]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [27]:
def binary_accuracy(preds1, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    preds, ind= torch.max(F.softmax(preds1),1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.truth)
        
        acc = binary_accuracy(predictions, batch.truth)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.truth)
            
            acc = binary_accuracy(predictions, batch.truth)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 20
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), dataset_path+'model-cnn.pt')
    #test_loss, test_acc = evaluate(model, test_iterator, criterion)

    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


| Epoch: 01 | Train Loss: 1.549 | Train Acc: 30.47% | Val. Loss: 1.500 | Val. Acc: 35.74
| Epoch: 02 | Train Loss: 1.447 | Train Acc: 38.81% | Val. Loss: 1.422 | Val. Acc: 39.27
| Epoch: 03 | Train Loss: 1.355 | Train Acc: 43.36% | Val. Loss: 1.362 | Val. Acc: 40.21
| Epoch: 04 | Train Loss: 1.276 | Train Acc: 47.38% | Val. Loss: 1.332 | Val. Acc: 41.10
| Epoch: 05 | Train Loss: 1.213 | Train Acc: 50.83% | Val. Loss: 1.305 | Val. Acc: 42.17
| Epoch: 06 | Train Loss: 1.156 | Train Acc: 54.12% | Val. Loss: 1.290 | Val. Acc: 42.62
| Epoch: 07 | Train Loss: 1.101 | Train Acc: 56.64% | Val. Loss: 1.274 | Val. Acc: 44.22
| Epoch: 08 | Train Loss: 1.054 | Train Acc: 60.05% | Val. Loss: 1.272 | Val. Acc: 43.56
| Epoch: 09 | Train Loss: 0.995 | Train Acc: 63.76% | Val. Loss: 1.266 | Val. Acc: 44.18
| Epoch: 10 | Train Loss: 0.948 | Train Acc: 66.35% | Val. Loss: 1.261 | Val. Acc: 45.16
| Epoch: 11 | Train Loss: 0.896 | Train Acc: 68.63% | Val. Loss: 1.259 | Val. Acc: 44.94
| Epoch: 12 | Train L

Test accuracy was **45.80%**

In [28]:
model.load_state_dict(torch.load(dataset_path+'model-cnn.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.200 | Test Acc: 45.80%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """
